# Downloading DEM data

In [47]:
%load_ext jupyter_black
from bs4 import BeautifulSoup
import requests
from pathlib import Path
import os
import geopandas as gpd

The jupyter_black extension is already loaded. To reload it, use:
  %reload_ext jupyter_black


In [48]:
input_dir = (
    Path(os.getenv("STORM_DATA_DIR"))
    / "analysis/02_new_model_input/04_topography/input/srtm/"
)
shp_input_dir = (
    Path(os.getenv("STORM_DATA_DIR"))
    / "analysis/02_new_model_input/02_housing_damage/input/"
)

In [32]:
base_url = "https://srtm.csi.cgiar.org/wp-content/uploads/files/srtm_5x5/TIFF/"
ext = "zip"
page = requests.get(base_url, verify=False).text
soup = BeautifulSoup(page, "html.parser")
for node in soup.find_all("a"):
    if node.get("href").endswith(ext):
        file_url = [
            base_url + "/" + node.get("href")
            for node in soup.find_all("a")
            if node.get("href").endswith(ext)
        ]
        file_list = [
            node.get("href")
            for node in soup.find_all("a")
            if node.get("href").endswith(ext)
        ]

c:\Users\pauni\Desktop\Work\OCHA\GitHub\VirtualEnv\globaltyphoon\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'srtm.csi.cgiar.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Using country extent to get which boxes to download.

In [49]:
adm3_shp = gpd.read_file(
    shp_input_dir / "phl_adminboundaries_candidate_adm3.zip"
)

In [126]:
phl_extent = list(adm3_shp.total_bounds)
wld_lon_extent = 180 + 180  # 180W to 180E
wld_lat_extent = 60 + 60  # 60N to 60S
wld_lon_box = wld_lon_extent / 72  # 72 is the number of columns of data
wld_lat_box = wld_lat_extent / 24  # 24 is the number of rows of data
wld_lon_start = list(range(-180, 180, int(wld_lon_box)))
wld_lat_start = list(range(60, -60, int(wld_lat_box) * -1))
phl_lat_start = [n for n, i in enumerate(wld_lat_start) if i < phl_extent[3]][
    0
]
phl_lat_end = [n for n, i in enumerate(wld_lat_start) if i > phl_extent[1]][
    -1:
][0]
phl_lon_start = [n for n, i in enumerate(wld_lon_start) if i > phl_extent[0]][
    0
]
phl_lon_end = [n for n, i in enumerate(wld_lon_start) if i > phl_extent[2]][0]
lat_list = list(range(phl_lat_start, phl_lat_end + 1))
lon_list = list(range(phl_lon_start, phl_lon_end + 1))
phl_list = ["%02d" % x + "_" + "%02d" % y for x in lon_list for y in lat_list]
phl_file_list = ["srtm_" + file + ".zip" for file in phl_list]

In [128]:
for file in phl_file_list:
    req = requests.get(base_url + file, verify=False, stream=True)
    with open(input_dir / file, "wb") as output_file:
        output_file.write(req.content)

c:\Users\pauni\Desktop\Work\OCHA\GitHub\VirtualEnv\globaltyphoon\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'srtm.csi.cgiar.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\pauni\Desktop\Work\OCHA\GitHub\VirtualEnv\globaltyphoon\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'srtm.csi.cgiar.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\pauni\Desktop\Work\OCHA\GitHub\VirtualEnv\globaltyphoon\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'srtm.csi.cgiar.org'. Adding certificate verification is strongly advised. See: https://urllib3.read